In [1]:
import os
from os.path import join as opj
import numpy as np

import open3d as o3d
from open3d.web_visualizer import draw

from utils import *
from dataset import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
# set_random_seed(0)

data = dict(
    data_root='data_root',
    partial=False,
    category=['grasp', 'contain', 'lift', 'openable', 'layable', 'sittable',
              'support', 'wrap_grasp', 'pourable', 'move', 'displaY', 'pushable', 'pull',
              'listen', 'wear', 'press', 'cut', 'stab']
)

data_root = data['data_root']
partial = data['partial']
aff_train_set = AffordNetDataset(data_root, 'train', partial=partial)
aff_val_set = AffordNetDataset(data_root, 'val', partial=partial)
print("Loaded dataset with {} training samples and {} validation samples".format(len(aff_train_set), len(aff_val_set)))


Loaded dataset with 16082 training samples and 2285 validation samples


In [9]:
classes_of_interest = ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
# pick a random datapoint belonging to a class of interest
while(1):
    rand_index = np.random.randint(0, len(aff_train_set.all_data))#9532
    data_temp = aff_train_set.all_data[rand_index]
    if data_temp['semantic class'] in classes_of_interest:
        print("Chosen random index: ", rand_index)
        break

# # pick a specific shape
# for item in aff_train_set.all_data:
#     if item['shape_id'] == 'fa8124cce25881004fcdbc2114c724fa':
#         data_temp = item
#         break

In [10]:
print("Shape id ", data_temp['shape_id'], "\nSemantic class: ", data_temp['semantic class'])
print("All affordances: ", data_temp["affordance"])
print("Num affordances: ", len(data_temp["affordance"]))

Shape id  fa8124cce25881004fcdbc2114c724fa 
Semantic class:  Hat
All affordances:  ['grasp', 'contain', 'lift', 'openable', 'layable', 'sittable', 'support', 'wrap_grasp', 'pourable', 'move', 'displaY', 'pushable', 'pull', 'listen', 'wear', 'press', 'cut', 'stab']
Num affordances:  18


In [11]:
pcl = data_temp['data_info']['coordinate']
# Viz pcl with o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
# color black
pcd.paint_uniform_color([0, 0, 0])
# viz in jupyter nb
print("Drawing pcd with", len(pcl), "points")
draw(pcd)

Drawing pcd with 2048 points
[Open3D INFO] Window window_5 created.


WebVisualizer(window_uid='window_5')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.8558459078223097
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.8558459078223097
[Open3D INFO] Sending init frames to window_5.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate


In [12]:
# get pcls of all labels
labels_list = []
scores_list = []
for key in data_temp['data_info']['label'].keys():
    labels = data_temp['data_info']['label'][key]
    scores = labels.sum()
    print("Label: ", key, " Score: ", scores)
    labels_list.append(labels)
    scores_list.append(scores)

Label:  grasp  Score:  220.94138
Label:  contain  Score:  0.0
Label:  lift  Score:  0.0
Label:  openable  Score:  0.0
Label:  layable  Score:  0.0
Label:  sittable  Score:  0.0
Label:  support  Score:  0.0
Label:  wrap_grasp  Score:  0.0
Label:  pourable  Score:  0.0
Label:  move  Score:  0.0
Label:  displaY  Score:  0.0
Label:  pushable  Score:  0.0
Label:  pull  Score:  0.0
Label:  listen  Score:  0.0
Label:  wear  Score:  272.82758
Label:  press  Score:  0.0
Label:  cut  Score:  0.0
Label:  stab  Score:  0.0


In [13]:
labels_list = np.array(labels_list)
scores_list = np.array(scores_list)
# Get max 4 labels and viz
best_score_ids = np.argsort(scores_list)[-4:]
# reverse order
best_score_ids = best_score_ids[::-1]
print("Best labels: ")
for best_score_id in best_score_ids:
    print(list(data_temp['data_info']['label'].keys())[best_score_id])
print("Best scores: ", scores_list[best_score_ids])

Best labels: 
wear
grasp
wrap_grasp
contain
Best scores:  [272.82758 220.94138   0.        0.     ]


In [ ]:
pcd_0 = o3d.geometry.PointCloud()
pcd_0.points = o3d.utility.Vector3dVector(pcl)
pcd_0.paint_uniform_color([0, 0, 0])
np.asarray(pcd_0.colors)[labels_list[best_score_ids[0]].squeeze() > 0] = [1, 0, 0]
print("Label 0: ", list(data_temp['data_info']['label'].keys())[best_score_ids[0]])
draw([pcd_0], title="Label: " + list(data_temp['data_info']['label'].keys())[best_score_ids[0]])

pcd_1 = o3d.geometry.PointCloud()
pcd_1.points = o3d.utility.Vector3dVector(pcl)
pcd_1.paint_uniform_color([0, 0, 0])
np.asarray(pcd_1.colors)[labels_list[best_score_ids[1]].squeeze() > 0] = [0, 1, 0]
print("Label 1: ", list(data_temp['data_info']['label'].keys())[best_score_ids[1]])
draw([pcd_1], title="Label: " + list(data_temp['data_info']['label'].keys())[best_score_ids[1]])

pcd_2 = o3d.geometry.PointCloud()
pcd_2.points = o3d.utility.Vector3dVector(pcl)
pcd_2.paint_uniform_color([0, 0, 0])
np.asarray(pcd_2.colors)[labels_list[best_score_ids[2]].squeeze() > 0] = [0, 0, 1]
print("Label 2: ", list(data_temp['data_info']['label'].keys())[best_score_ids[2]])
draw([pcd_2], title="Label: " + list(data_temp['data_info']['label'].keys())[best_score_ids[2]])

pcd_3 = o3d.geometry.PointCloud()
pcd_3.points = o3d.utility.Vector3dVector(pcl)
pcd_3.paint_uniform_color([0, 0, 0])
np.asarray(pcd_3.colors)[labels_list[best_score_ids[3]].squeeze() > 0] = [1, 1, 0]
print("Label 3: ", list(data_temp['data_info']['label'].keys())[best_score_ids[3]])
draw([pcd_3], title="Label: " + list(data_temp['data_info']['label'].keys())[best_score_ids[3]])

Label 0:  wear
[Open3D INFO] Window window_6 created.


WebVisualizer(window_uid='window_6')

Label 1:  grasp
[Open3D INFO] Window window_7 created.


WebVisualizer(window_uid='window_7')

Label 2:  wrap_grasp
[Open3D INFO] Window window_8 created.


WebVisualizer(window_uid='window_8')

Label 3:  contain
[Open3D INFO] Window window_9 created.


WebVisualizer(window_uid='window_9')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6691501978407153
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.10353951660444305
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.669